In [ ]:
import pandas as pd
import csv
import json
import random
from openai import OpenAI
import requests

In [ ]:
json_file_path = r'C:\Workspaces\CodeSpaces\Python_Work\ChatGpt_AzureAI\21_12_23\Mapping.json'
# Read raw JSON data
with open(json_file_path, 'r') as json_file_path:
    json_data = json.load(json_file_path)
# Print the raw JSON data
print(json.dumps(json_data, indent=2)) # Use indent for pretty printing


In [23]:
# input_data_df = pd.read_csv('input_data.csv')
input_csv = r'C:\Workspaces\CodeSpaces\Python_Work\ChatGpt_AzureAI\21_12_23\test.csv'
with open(input_csv, 'r') as input_file:
    reader = csv.reader(input_file)
    # Iterate through the rows and print each row
    for row in reader:
        print(row)

['MGA', 'Issuing Company', 'Full policy number', 'Insured Name']
['Arrowhead General Insurance Agency Inc', 'TSIC', 'TSAHGL0000025-00', 'Rego Realty Corporation']
['Arrowhead General Insurance Agency Inc', 'TSIC', 'TSAHGL0000050-00', 'J Sterling Quality Roofing, Inc.']
['Arrowhead General Insurance Agency Inc', 'TSIC', 'TSAHGL0000050-00', 'J Sterling Quality Roofing, Inc.']
['Plimsoll Specialty Markets LLC', 'Transverse Specialty Insurance Company', 'TSBAAC 0000002-00', '3D Aviation Inc']
['Marsh USA Chicago, Inc., ', 'Transverse Specialty Insurance Company', 'TSBAAC 0000091-00', 'Venture Travel, LLC']
['Marsh USA Chicago, Inc., ', 'Transverse Specialty Insurance Company', 'TSBAAG 0000092-00', 'Venture Travel, LLC']
['Marsh & McLennan Agency', 'Transverse Specialty Insurance Company', 'TSBAAC 0000086-00', 'Gateway Canyons Air Tours, LLC']
['Marsh & McLennan Agency', 'Transverse Specialty Insurance Company', 'TSBAAC 0000086-00', 'Gateway Canyons Air Tours, LLC']
['Travers & Associates',

In [ ]:
# import pandas as pd

# file_path = r'C:\Workspaces\CodeSpaces\Python_Work\ChatGpt_AzureAI\21_12_23\Mapping.json'

# # Read JSON data into a DataFrame
# df = pd.read_json(file_path, orient='records')  # Set 'orient' based on your JSON structure

# # Print the DataFrame
# print(df)


In [ ]:
# file_path = r'C:\Workspaces\CodeSpaces\Python_Work\ChatGpt_AzureAI\21_12_23\Mapping.json'

# def read_csv(file_path):
#     return pd.read_json(file_path, index_col=False)

In [ ]:
def create_pairs(df, alt_names):
    pairs = []
    confidence_scores = [100, 0]

    for ind, column1 in enumerate(df.columns):
        i = 0
        for index, column in enumerate(df.columns):
            if column1 == column:
                if column1 == column:
                    prompt = f"""Your task is to classify and return a confidence score for the {column} column with the {column1} column. This is your target column: {column1}. If the {column} column name is similar to {alt_names[ind]}, then the confidence score is {confidence_scores[i]}. Also, if the data patterns, data types, common values, or trends in the {column} column ({list(df.iloc[:, index].values)}) are similar to {list(df.iloc[:, ind].values)} data, then the confidence score is {confidence_scores[i]}. Now, based on the analysis, generate the output\n\n##\n\n"""
                    completion = f"""{confidence_scores[i]},{column}, {column1} \n\n##\n\n"""
                    pairs.append({"prompt": prompt, "completion": completion})
                else:
                    prompt = f"""Your task is to classify and return a confidence score for the {column} column with the {column1} column. This is your target column: {column1}. If the {column} column name is not similar to {alt_names[index]}, then the confidence score is {confidence_scores[i+1]}. Also, if the data patterns, data types, common values, or trends in the {column} column ({list(df.iloc[:, index].values)}) are not similar to {list(df.iloc[:, ind].values)} data, then the confidence score is {confidence_scores[i+1]}. Now, based on the analysis, generate the output\n\n##\n\n"""
                    completion = f"""{confidence_scores[i+1]},{column} , {column1}\n\n##\n\n"""
                    pairs.append({"prompt": prompt, "completion": completion})
    return pairs

In [ ]:
def save_pairs_to_json(pairs, file_path):
    random.shuffle(pairs)
    with open(file_path, 'w') as f:
        json.dump(pairs, f)

In [ ]:
input_data_df = pd.read_csv('input_data.csv')
alt_names_df = pd.read_csv('alternative.csv')

In [ ]:
alt_names = alt_names_df.iloc[:, 1].tolist()

# Create pairs
pairs = create_pairs(input_data_df, alt_names)

# Save pairs to JSON
save_pairs_to_json(pairs, "new_prompt_64.json")

In [ ]:
api_key = 'your_api_key'  # Replace with your actual API key
client = OpenAI(api_key=api_key)

try:
    response = client.files.create(
        file=open("new_prompt_64.json", "rb"),
        purpose="fine-tune"
    )
    file_id = response.id
    print("File ID:", file_id)
    print(response)
except Exception as e:
    print(f"Error during file upload: {e}")

In [ ]:
model_id = response.model

# Create a fine-tuning job
url = "https://api.openai.com/v1/fine_tuning/jobs"
headers = {"Content-Type": "application/json", "Authorization": f"Bearer {api_key}"}
payload = {"training_file": file_id, "model": model_id}

try:
    response = requests.post(url, headers=headers, json=payload)
    job_id = response.json()["id"]
    print("Fine-tuning job created. Job ID:", job_id)
except Exception as e:
    print(f"Error creating fine-tuning job: {e}")

# Wait for the fine-tuning job to complete
while True:
    url = f"https://api.openai.com/v1/fine_tuning/jobs/{job_id}"
    try:
        response = requests.get(url, headers=headers)
        status = response.json()["status"]
        print("Fine-tuning status:", status)
        if status == "succeeded":
            break  # Exit the loop if fine-tuning is complete
        elif status == "failed":
            print("Fine-tuning job failed.")
            break
    except Exception as e:
        print(f"Error checking fine-tuning status: {e}")

In [ ]:
model = f"ft:{model_id}"
headers = {"Content-Type": "application/json", "Authorization": f"Bearer {api_key}"}

In [ ]:
test_data = r"C:\Workspaces\CodeSpaces\Python_Work\ChatGpt_AzureAI\21_12_23\test.csv"
benchmark = r"C:\Workspaces\CodeSpaces\Python_Work\ChatGpt_AzureAI\21_12_23\21_12_8.csv"
test_df = pd.read_csv(test_data)
bench = pd.read_csv(benchmark)

In [ ]:
for column in test_df.columns:
    for col in bench.columns:
        payload = {
            "prompt": f"""Your task is to classify and return a confidence_score for the given {column} column with the {col} column. This is your target column: {col}. {column} data is {list(test_df[column].values)} and {col} data is {list(bench[col].values)}.Now, based on the analysis, generate the output\n\n##\n\n""",
            "temperature": 0.02,
            "max_tokens": 15,
        }

        try:
            response = requests.post(f"https://api.openai.com/v1/engines/{model}/completions", headers=headers, json=payload)
            result = response.json()
            confidence_score = result["choices"][0]["text"].split(',')[0].strip()
            print(f"{confidence_score}, {column}, {col}")
        except Exception as e:
            print(f"Error generating confidence score: {e}")
